# 📘 05강 — 모나드 & 컨테이너 추상화

3강(FlatMap·Maybe/Optional), 4강(함수 합성 & 파이프라인)을 토대로 **에러·결측·검증**을 합성 가능한 체인으로 다룹니다.


---

### 🎯 학습목표
- `Functor(map)`와 `Monad(and_then/flatmap)`의 차이를 이해하고 직접 구현한다.
- `Maybe(Just/Nothing)`와 `Result(Ok/Err)`를 **일관된 인터페이스**로 다룬다.
- 검증 → 변환 → 집계로 이어지는 **합성 가능한 체인**을 구축한다.
- 모나드 법칙(좌/우 항등, 결합법칙)을 작은 예제로 감각적으로 점검한다.

### 🧩 키워드
`Functor`, `Monad`, `map`, `flatmap`, `and_then`, `Kleisli composition`, `Maybe`, `Result`, `sequence`, `traverse`


---

## 🧩 Functor — 값을 올려 태우기 (`map`)
- 컨테이너 `F[T]` 위에 `T -> U` 함수를 올려 `F[U]`로 변환
- 실패/결측 상태라면 **통과(no-op)**가 바람직


In [13]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Protocol, Callable, Generic, TypeVar

T = TypeVar("T")
U = TypeVar("U")

class Functor(Protocol[T]):
    def map(self, f: Callable[[T], U]) -> "Functor[U]": ...

# 개념 데모용 Box Functor
@dataclass(frozen=True, slots=True)
class Box(Generic[T]):
    value: T
    def map(self, f: Callable[[T], U]) -> "Box[U]":
        return Box(f(self.value))

Box(10).map(lambda x: x + 5)  # Box(15)


Box(value=15)

---

## 🧩 Monad — 안전한 연결 (`and_then` / `flatmap`)
- `T -> F[U]`를 **연결(체이닝)** 하여 `F[T] -> F[U]`
- 중간에 실패/결측이 나오면 **단락(short-circuit)**


In [14]:
from typing import Protocol, Callable

class Monad(Functor[T], Protocol[T]):
    def and_then(self, f: Callable[[T], "Monad[U]"]) -> "Monad[U]": ...


---

## 🧩 Maybe — 결측을 표현 (Just / Nothing)
- 값이 없을 수 있는 경우를 **명시적으로** 표현
- `map`/`and_then`는 `Nothing`에서 **무시**되어 안전


In [15]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Callable, Generic, TypeVar

T = TypeVar("T")
U = TypeVar("U")

@dataclass(frozen=True, slots=True)
class Just(Generic[T]):
    value: T
    def map(self, f: Callable[[T], U]) -> "Just[U]":
        return Just(f(self.value))
    def and_then(self, f: Callable[[T], "Just[U] | Nothing[U]"]) -> "Just[U] | Nothing[U]":
        return f(self.value)
    def get_or(self, default: T) -> T:
        return self.value
    def __repr__(self) -> str:
        return f"Just({self.value!r})"

class Nothing(Generic[T]):
    __slots__ = ()
    def map(self, f: Callable[[T], U]) -> "Nothing[U]":
        return self
    def and_then(self, f: Callable[[T], "Just[U] | Nothing[U]"]) -> "Nothing[U]":
        return self
    def get_or(self, default: T) -> T:
        return default
    def __repr__(self) -> str:
        return "Nothing"

NOTHING: Nothing[Any] = Nothing()

def maybe(v: T | None) -> Just[T] | Nothing[T]:
    return Just(v) if v is not None else NOTHING  # type: ignore[return-value]

# 데모
maybe(10).map(lambda x: x*2)         # Just(20)
maybe(None).map(lambda x: x*2)       # Nothing


Nothing

---

## 🧩 Result — 성공/실패를 분리 (Ok / Err)
- 실패의 **이유(메시지/코드)**를 보존
- `map`은 성공값만 변환, `map_err`는 실패값만 변환


In [16]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Callable, Generic, TypeVar

T = TypeVar("T")
U = TypeVar("U")
E = TypeVar("E")

@dataclass(frozen=True, slots=True)
class Ok(Generic[T]):
    value: T
    def map(self, f: Callable[[T], U]) -> "Ok[U]":
        return Ok(f(self.value))
    def and_then(self, f: Callable[[T], "Ok[U] | Err[E]"]) -> "Ok[U] | Err[E]":
        return f(self.value)
    def get_or(self, default: T) -> T:
        return self.value
    def map_err(self, g: Callable[[E], Any]) -> "Ok[T]":
        return self
    def get_or_raise(self) -> T:
        return self.value
    def __repr__(self) -> str:
        return f"Ok({self.value!r})"

@dataclass(frozen=True, slots=True)
class Err(Generic[E]):
    error: E
    def map(self, f: Callable[[Any], Any]) -> "Err[E]":
        return self
    def and_then(self, f: Callable[[Any], Any]) -> "Err[E]":
        return self
    def get_or(self, default: T) -> T:
        return default
    def map_err(self, g: Callable[[E], U]) -> "Err[U]":
        return Err(g(self.error))
    def get_or_raise(self) -> Any:
        raise RuntimeError(self.error)
    def __repr__(self) -> str:
        return f"Err({self.error!r})"

Result = Ok[T] | Err[E]

def ok(v: T) -> Ok[T]: return Ok(v)
def err(e: E) -> Err[E]: return Err(e)

# 간단 데모
ok(3).map(lambda x: x+1)           # Ok(4)
err("boom").map(lambda x: x+1)     # Err("boom")


Err('boom')

---

## 🧪 모나드 법칙 — 감각적 점검 (Result)
- 좌항등: `unit(x) >>= f == f(x)`
- 우항등: `m >>= unit == m`
- 결합법칙: `(m >>= f) >>= g == m >>= (λx. f(x) >>= g)`


In [17]:
from typing import Callable

def f(x: int) -> Result[int, str]: return ok(x + 1)
def g(x: int) -> Result[int, str]: return ok(x * 2)
unit: Callable[[int], Result[int, str]] = ok
m: Result[int, str] = ok(7)

# 좌항등
assert unit(10).and_then(f) == f(10)
# 우항등
assert m.and_then(unit) == m
# 결합법칙
left  = m.and_then(f).and_then(g)
right = m.and_then(lambda v: f(v).and_then(g))
assert left == right

print("모나드 법칙 체크: OK")


모나드 법칙 체크: OK


---

## 🧩 Kleisli 합성 & 공통 유틸
여러 `A -> Result[B]` 함수를 **하나의 화살표**로 합성합니다.


In [18]:
from typing import TypeVar, Iterable, Callable

A = TypeVar("A")
B = TypeVar("B")
C = TypeVar("C")

def compose_result(g: Callable[[B], Result[C, E]], f: Callable[[A], Result[B, E]]) -> Callable[[A], Result[C, E]]:
    def h(x: A) -> Result[C, E]:
        return f(x).and_then(g)
    return h

# 공통 유틸
def parse_int(s: str) -> Result[int, str]:
    try:
        return ok(int(s))
    except ValueError:
        return err(f"'{s}'는 int로 변환 불가")

def validate_positive(n: int) -> Result[int, str]:
    return ok(n) if n > 0 else err("양수여야 합니다")

def reciprocal(n: int) -> Result[float, str]:
    return ok(1.0 / n) if n != 0 else err("0으로 나눌 수 없습니다")

# 간단 체인
ok(" 42 ").map(str.strip).and_then(parse_int).and_then(validate_positive).and_then(reciprocal)


Ok(0.023809523809523808)

---

## 🔧 도메인 파이프라인 — 가입 폼 예시
검증 → 정규화 → 엔티티화 과정을 `and_then` 체인과 Kleisli로 모두 표현합니다.


In [19]:
from dataclasses import dataclass
from typing import Callable

@dataclass(frozen=True, slots=True)
class User:
    email: str
    age: int

def require_keys(keys: set[str]) -> Callable[[dict[str, object]], Result[dict[str, object], str]]:
    def _check(d: dict[str, object]) -> Result[dict[str, object], str]:
        missing = [k for k in keys if k not in d]
        return err(f"누락 필드: {missing}") if missing else ok(d)
    return _check

def normalize(d: dict[str, object]) -> Result[dict[str, object], str]:
    try:
        email = str(d["email"]).strip().lower()
        age = int(d["age"])
        return ok({"email": email, "age": age})
    except Exception as e:
        return err(f"정규화 실패: {e}")

def validate_email(d: dict[str, object]) -> Result[dict[str, object], str]:
    email = d["email"]
    return ok(d) if isinstance(email, str) and "@" in email else err("이메일 형식 오류")

def validate_age(d: dict[str, object]) -> Result[dict[str, object], str]:
    age = d["age"]
    return ok(d) if isinstance(age, int) and (14 <= age <= 120) else err("나이 범위 오류")

def to_user(d: dict[str, object]) -> User:
    return User(email=d["email"], age=d["age"])  # type: ignore[arg-type]

# 체인
def build_user_chain(payload: dict[str, object]) -> Result[User, str]:
    return (
        ok(payload)
        .and_then(require_keys({"email", "age"}))
        .and_then(normalize)
        .and_then(validate_email)
        .and_then(validate_age)
        .map(to_user)
    )

# Kleisli
pipeline = compose_result(validate_age,
           compose_result(validate_email,
           compose_result(normalize,
                          require_keys({"email", "age"}))))

def build_user_kleisli(payload: dict[str, object]) -> Result[dict[str, object], str]:
    return pipeline(payload)

print("chain  :", build_user_chain({"email":"  A@B.C ", "age":"23"}))
print("kleisli:", build_user_kleisli({"email":"  A@B.C ", "age":"23"}))
print("bad1   :", build_user_chain({"email":"no-at","age":20}))
print("bad2   :", build_user_chain({"age":20}))


chain  : Ok(User(email='a@b.c', age=23))
kleisli: Ok({'email': 'a@b.c', 'age': 23})
bad1   : Err('이메일 형식 오류')
bad2   : Err("누락 필드: ['email']")


---

## 🔧 실습 — `sequence` / `traverse` (실행 가능 버전)
여러 `Result`를 **한 번에** 다루는 일반화 패턴입니다.


In [25]:
from functools import reduce
from typing import Iterable, Callable, TypeVar

T = TypeVar("T")
U = TypeVar("U")
E = TypeVar("E")
A = TypeVar("A")
B = TypeVar("B")

def sequence_results(xs: Iterable[Result[T, E]]) -> Result[list[T], E]:
    def merge(acc: Result[list[T], E], r: Result[T, E]) -> Result[list[T], E]:
        if isinstance(acc, Err): return acc
        if isinstance(r, Err):   return r
        v = acc.value.copy()
        v.append(r.value)
        return ok(v)
    return reduce(merge, xs, ok([]))

def traverse_results(xs: Iterable[A], f: Callable[[A], Result[B, E]]) -> Result[list[B], E]:
    return sequence_results(map(f, xs))

# 데모
data_good = ["1", "2", "3"]
data_bad  = ["1", "x", "3"]
print(traverse_results(data_good, parse_int))  # Ok([1, 2, 3])
print(traverse_results(data_bad, parse_int))   # Err("'x'는 int로 변환 불가")

# 간단 확인
assert sequence_results([ok(1), ok(2)]) == ok([1, 2])
assert isinstance(sequence_results([ok(1), err("boom")]), Err)
assert traverse_results(["10","20"], parse_int) == ok([10, 20])

print("실습 구현: OK")


Ok([1, 2, 3])
Err("'x'는 int로 변환 불가")
실습 구현: OK


## ⚡ 정리
- **Functor/Monad**는 *함수를 안전하게 올려 태우고 연결*하는 공통 인터페이스를 제공합니다.
- `Maybe`는 **없음(None)**을, `Result`는 **실패의 이유**까지 표현합니다.
- `and_then` 체인은 **단락(short-circuit)**으로 복잡한 if/try 중첩을 제거합니다.
- **Kleisli 합성**으로 `A -> Result[B] -> Result[C]`를 한 화살표로 합성합니다.
- `sequence`/`traverse`는 **여러 개의 결과를 한 번에** 다룰 수 있게 일반화합니다.

## 📝 Assignment
1) `Result`에 **`apply` (Applicative)**를 추가하세요.  
   - 시그니처: `apply(self, rf: Result[Callable[[T], U], E]) -> Result[U, E]`  
   - 목적: `Result` 안의 함수를 `Result` 안의 값에 적용 (여러 검증을 병렬처럼 표현)
2) `Maybe` ↔ `Result` 변환 유틸을 작성하세요.  
   - `maybe_to_result(m: Just[T] | Nothing[T], error: E) -> Result[T, E]`  
   - `result_to_maybe(r: Result[T, E]) -> Just[T] | Nothing[T]`
3) `sequence`/`traverse`를 사용해 **폼 배열**(딕셔너리 리스트)을 일괄 검증하는 파이프라인을 만드세요.  
   - 성공: `Ok[List[User]]`, 실패: 첫 에러로 `Err[str]`

## 규칙
- 함수는 **순수(pure)** 하게 작성하고, I/O는 다루지 않습니다.
- `map`/`and_then`(= `flatmap`) 중심으로 합성하세요.
- 불필요한 `try/except` 중첩, 과도한 명령형 분기 지양.
- `@dataclass(frozen=True, slots=True)` + **타입힌트** 사용.
- 표준 라이브러리만 사용합니다.
